In [1]:
from bs4 import BeautifulSoup
import requests
import json,codecs
import pandas as pd
import re
import os
import time


def request_url(url,method="GET",data=None):
    # เพิ่มส่วนหัว (Headers) เพื่อเลียนแบบเบราว์เซอร์
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
    }
    response = None
    if method == "GET":
        response = requests.get(url, headers=headers)
    else:
        # กำหนด headers ด้วย Content-Type เป็น application/json
        headers["Content-Type"] = "application/json"

        # ส่งคำขอ POST
        response = requests.post(url, data=json.dumps(data), headers=headers)      

    # ตรวจสอบสถานะการตอบกลับ
    if response.status_code == 200:
        return response
    elif response.status_code == 403:
        print("Access is forbidden. Try modifying headers or check if the server blocks scraping.")
        return False
    elif response.status_code == 404:
        print("Page not found. Check the URL.")
        return False
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        print(f"Failed to retrieve the webpage. Status code: {response.reason}")
        return False

# อ่านไฟล์ JSON
file_path = 'data/flood-index-thairath.json'

with open(file_path, 'r', encoding='utf-8') as file:
    news = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python

for index, item in enumerate(news):
    directory = 'data-detail'
    filenames = os.listdir(directory)
    
    news_id = item["tags"][-1]
    if f"{news_id}.json" in filenames : 
        # print(f"SKIP : {news_id}")
        continue

    # Step 1: ระบุ URL ของหน้าเว็บ
    url = item["url"]
    # url = "https://www.thairath.co.th/news/local/south/2831171"
    # url = "https://cs.vru.ac.th/"

    # Step 2: ใช้ requests เพื่อดึง HTML
    response = request_url(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # print(soup.prettify())
    div1 = soup.select("div.__item_article-headline")[0]
    div2 = soup.select("div.__item_article-content")[0]

    title = div1.text
    content =  div2.text
    print(len(title),len(content))
    print(title)
    print(content)

    # Step 3: ใช้ requests เพื่อดึง NER
    # URL ของ API
    url = "https://ml.ckartisan.com/ner/text"
    # ข้อมูลที่ต้องการส่งในรูปแบบ JSON
    data = {
        "content": f"{title} {content}",
    }
    response = request_url(url,"POST",data)    
    if response == False:           
        print(f"500 from {news_id}")     
        print("Wait for 30 seconds && continue")
        time.sleep(30)
        continue
    ner = response.json()
    # data["data_length"] = len(data["original_message"])
    # print(ner)
    item["ner_obj"] = ner

    news_id = item["tags"][-1]
    filename = f"data-detail/{news_id}.json"
    with codecs.open(filename, 'w', encoding='utf-8') as f:
        json.dump(item, f, ensure_ascii=False)
        print(f"Write file {filename}")
    
    print(f"DONE: {index+1}/{len(news)} : {round((index+1)/len(news)*100,0)}%")

    
    time.sleep(5)   # cool down 5 seconds
    # exit from Loop
    # break



78 3969
‘เจ้าพระยา’ ดีขึ้น น้ำจากเหนือน้อยลง ภาคใต้ยังอ่วม เตือนฝนถล่ม 9 จังหวัดรับมือ
ศปช.เตือน 9 จังหวัดภาคใต้รับมือฝนตกหนัก ดินถล่ม ขณะที่ สถานการณ์จังหวัดลุ่มน้ำเจ้าพระยามีแนวโน้มดีขึ้น หลังน้ำเหนือมีปริมาณน้อยลง เขื่อนเจ้าพระยาปรับลดระบายน้ำเหลือ 1,650 ลบ.ม./วินาที ส่งผลดีต่อจังหวัดท้ายเขื่อน ส่วนที่เชียงใหม่จิตอาสาทุกภาคส่วนรวมพลังปล่อยคาราวานเก็บขยะล้างทำความสะอาดเมืองครั้งใหญ่ ขณะที่ช่างซ่อมมอเตอร์ไซค์จากร้านต่างๆรวมตัวเปิดซ่อมรถจมน้ำให้ชาวบ้านฟรีกว่า 100 คันศปช.เตือน 9 จังหวัดภาคใต้ ฝนตกหนักและดินสไลด์ เมื่อเวลา 13.00 น. วันที่ 14 ต.ค. นายจิรายุ ห่วงทรัพย์ โฆษกศูนย์ปฏิบัติการช่วยเหลือผู้ประสบอุทกภัย วาตภัย และดินโคลนถล่ม (ศปช.) เปิดเผยว่า จากปริมาณฝนยังตกหนักถึงหนักมากในภาคใต้ อาจส่งผลให้เกิดเหตุดินถล่มใน 9 จังหวัด ได้แก่ จ.ภูเก็ต อ.เมืองภูเก็ต อ.กะทู้ อ.ถลาง, จ.กระบี่ อ.เมือง กระบี่ อ.เขาพนม อ.เกาะลันตา, จ.ตรัง อ.ย่านตาขาว อ.ปะเหลียน อ.ห้วยยอด อ.นาโยง, จ.สตูล อ.ควนโดน อ.ควนกาหลง อ.ละงู อ.ทุ่งหว้า อ.มะนัง, จ.พัทลุง อ.กงหรา อ.ศรีบรรพต อ.ศรีนครินทร์ อ.เขาชัยสน อ.ตะโหมด อ.ควนขนุน

In [8]:
text = "สวัสดีครับ ผมชื่อจอห์น วันนี้จะพาทุกคนไปเที่ยวที่ทะเล สวยมากๆ เลยครับ"

text[-1]


'บ'